## This notebook is to generate train and valid datasets.

In [1]:
import os
import sys
import copy
import pickle as pkl
from tqdm import tqdm
import re
import numpy as np
from collections import Counter

In [2]:

import pickle as pkl
import ujson as json
def save_to_disk(obj, file_name):
    print('saving ', file_name)
    with open(file_name, 'wb') as f:
        pkl.dump(obj, f, protocol=pkl.HIGHEST_PROTOCOL)
 
def load_from_disk(file_name):
    print('loading ', file_name)
    with open(file_name, 'rb') as f:
        obj = pkl.load(f)
    return obj

def word_tokenize(sent):
    """Tokenize a sententence

    Args:
        sent: the sentence need to be tokenized

    Returns:
        list: words in the sentence
    """

    # treat consecutive words or special punctuation as words
    sent = sent.replace('\t', ' ')
    sent = sent.replace('<br/>', ' ')
    sent = sent.replace('\n', ' ')
    sent = re.sub(' +', ' ', sent)
    pat = re.compile(r"[\w]+|[.,!?;|]")
    if isinstance(sent, str):
        return pat.findall(sent.lower())
    else:
        return []





## 生成 news.tsv，用tab切割

In [3]:
word_dict_all_path = './data/word_dict_all.pkl'
word_dict_all = load_from_disk(word_dict_all_path)

loading  ./data/word_dict_all.pkl


In [4]:
paper_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/entities_paper.pkl'
project_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/entities_project.pkl'
projects = load_from_disk(project_path)
papers = load_from_disk(paper_path)


loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/entities_project.pkl
loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/entities_paper.pkl


In [5]:
# 获得 实体
# paper2wiki_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/NER/paper2wiki_0_-1.pkl'
# project2wiki_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/NER/project2wiki_0_-1.pkl'
# paper2wiki = load_from_disk(paper2wiki_path)
# project2wiki = load_from_disk(project2wiki_path)

loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/NER/paper2wiki_0_-1.pkl


In [ ]:
# def extract(wiki):
#     itself = []
#     for w in wiki:
#         tmp = {}
#         tmp['Label'] = w['wiki_label']
#         tmp['Type'] = 'P'
#         tmp['WikidataId'] = w['wiki_id']
#         tmp['Confidence'] = 1.0
#         tmp['OccurrenceOffsets'] = [w['pos'][0]]
#         tmp['SurfaceForms'] = w['text']
#         itself.append(tmp)
#     return itself
# project2wikiy = {}
# for p in project2wiki:
#     id = p['AwardID']
#     title = extract(p['AwardTitle']['wiki'])
#     abs = extract(p['AbstractNarration']['wiki'])
#     project2wikiy[id] = 

In [5]:
### 加载category
news2topic = {}

project2topic_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/LDA/project2topic.pkl'
f = load_from_disk(project2topic_path)

for d in f:
    news2topic[d[0]] = 'topic_project'+str(d[1])

paper2topic_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/LDA/paper2topic.pkl'
f = load_from_disk(paper2topic_path)
for d in f:
    news2topic[d[0]] = 'topic_paper'+str(d[1])

    


loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/LDA/project2topic.pkl
loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/LDA/paper2topic.pkl


In [7]:
len(news2topic)

2962215

In [10]:
news = []
for p in tqdm(projects):
    p = json.loads(p)
    title = '' if 'AwardTitle' not in p else p['AwardTitle']
    abs = '' if 'AbstractNarration' not in p else p['AbstractNarration']
    id = p['AwardID']
    category = 'null' if id not in news2topic else news2topic[id]
    entity_title = []
    entity_abs = []
    news_url = ''

    t = word_tokenize(title)
    t = [i for i in t if i in word_dict_all]
    title = ' '.join(t)

    t = word_tokenize(abs)
    t = [i for i in t if i in word_dict_all]
    abs = ' '.join(t)

    ss = f'{id}\t{category}\t{category}\t{title}\t{abs}\t{news_url}\t{json.dumps(entity_title)}\t{json.dumps(entity_abs)}'
    news.append(ss)


100%|██████████| 107125/107125 [00:22<00:00, 4841.61it/s]


In [11]:
for p in tqdm(papers):
    if len(p['authors']) == 0:
        continue
    title = '' if 'title' not in p else p['title']
    abs = '' if 'abstract' not in p else p['abstract']
    id = p['_id']
    category = 'null' if id not in news2topic else news2topic[id]
    entity_title = []
    entity_abs = []
    news_url = ''

    t = word_tokenize(title)
    t = [i for i in t if i in word_dict_all]
    title = ' '.join(t)

    t = word_tokenize(abs)
    t = [i for i in t if i in word_dict_all]
    abs = ' '.join(t)

    ss = f'{id}\t{category}\t{category}\t{title}\t{abs}\t{news_url}\t{json.dumps(entity_title)}\t{json.dumps(entity_abs)}'
    news.append(ss)

100%|██████████| 2855090/2855090 [01:25<00:00, 33461.65it/s]


In [13]:
with open('./data/news.tsv', 'w') as f:
    for new in news:
        f.write(new)
        f.write('\n')

## 生成 behaviors.tsv

In [15]:
publish_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/rel_is_publisher_of.pkl'
rel_publisher = load_from_disk(publish_path)
prin_invstor_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/train_rel_is_principal_investigator_of.pkl'
rel_prin_inves = load_from_disk(prin_invstor_path)

loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/rel_is_publisher_of.pkl
loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/train_rel_is_principal_investigator_of.pkl


In [19]:


from collections import defaultdict
import random
year2news = defaultdict(set)

person2news = {}
for line in rel_publisher:
    _, auid, newsid, year = line
    if year >= 2015:
        continue
    if auid not in person2news:
        person2news[auid] = defaultdict(set)
    person2news[auid][year].add(newsid)
    # year2news[year].add(newsid) 不需要做负样本

for line in rel_prin_inves:
    _, auid, newsid, year, _ = line
    if auid not in person2news:
        person2news[auid] = defaultdict(set)
    person2news[auid][year].add(newsid)
    year2news[year].add(newsid)

for y in year2news:
    year2news[y] = list(year2news[y])

behavior = []
for person in tqdm(person2news.keys()):
    collect = person2news[person]
    for year in collect.keys():
        if year < 2000:
            continue
        if year >= 2015:
            continue
        year_news = collect[year]
        neg = set()
        cnt = 0
        while len(neg) < len(year_news):
            neg_c = random.choice(year2news[year])
            if neg_c not in year_news:
                neg.add(neg_c)
            cnt += 1
            if cnt > len(year_news) * 2:
                break
        history = []
        for hy in collect:
            if hy < year:
                history += list(collect[hy])
        ss = (person, year, list(history), list(year_news), list(neg))
        behavior.append(ss)
print(len(behavior))


100%|██████████| 60138/60138 [00:02<00:00, 24723.67it/s]

290824


In [ ]:
hisc = []
impc = []
with open('./train/behaviors.tsv', 'w') as f:
    random.shuffle(behavior)
    for i, ss in enumerate(behavior):
        person, year, history, pos, neg = ss
        pos = [p+'-1' for p in pos if '-' not in p]
        neg = [n+'-0' for n in neg if '-' not in n]
        imp = pos + neg
        imp = ' '.join(imp)
        history = ' '.join(history)
        time = f'11/11/{year:04d} 3:28:58 PM'
        st = f'{i+1}\t{person}\t{time}\t{history}\t{imp}\n'
        f.write(st)
        hisc.append(len(history.split()))
        impc.append(len(imp.split()))
print(Counter(hisc))
print(Counter(impc))

In [ ]:
# test

In [29]:
publish_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/rel_is_publisher_of.pkl'
rel_publisher = load_from_disk(publish_path)
prin_invstor_path = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/test_rel_is_principal_investigator_of.pkl'
rel_prin_inves = load_from_disk(prin_invstor_path)

loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/rel_is_publisher_of.pkl
loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/test_rel_is_principal_investigator_of.pkl


In [32]:


from collections import defaultdict
import random
import numpy as np
year2news = defaultdict(set)

person2cnt = defaultdict(int)
person2news = {}
prin_invstor_path_train = '/home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/train_rel_is_principal_investigator_of.pkl'
rel_prin_inves_train = load_from_disk(prin_invstor_path_train)

for line in rel_publisher:
    _, auid, newsid, year = line
    if auid not in person2news:
        person2news[auid] = defaultdict(set)
    person2news[auid][year].add(newsid)
    # year2news[year].add(newsid) 不需要做负样本
    person2cnt[auid] += 1
val = [v for k,v in person2cnt.items()]
print(np.array(val).mean())

for line in rel_prin_inves_train:
    _, auid, newsid, year, _ = line
    if auid not in person2news:
        person2news[auid] = defaultdict(set)
    person2news[auid][year].add(newsid)
    year2news[year].add(newsid)

for y in year2news:
    year2news[y] = list(year2news[y])



behavior = []
cnt = 1
for line in rel_prin_inves:
    _, auid, newsid, year, negs = line
    history = []
    if auid in person2news:
        collect = person2news[auid]
        
        for y in collect.keys():
            if y >= year:
                continue
            year_news = collect[y]
            history += list(year_news)
    history = ' '.join(history)
    time = f'11/11/{year:04d} 3:28:58 PM'

    ss = f'{cnt}\t{auid}\t{time}\t{history}\t{newsid+"-1"}\n'
    behavior.append(ss)
    cnt += 1

    for neg in negs:
        
        history = []
        if neg in person2news:
            collect = person2news[neg]
            for y in collect.keys():
                if y >= year:
                    continue
                year_news = collect[y]
                history += list(year_news)
        history = ' '.join(history)
        time = f'11/11/{year:04d} 5:55:55 PM'

        ss = f'{cnt}\t{neg}\t{time}\t{history}\t{newsid+"-0"}\n'
        behavior.append(ss)
        cnt += 1


print(len(behavior))

with open('./valid/behaviors.tsv', 'w') as f:
    for ss in behavior:
        f.write(ss)


loading  /home/zy/data2/yyr/codes/kg4proj-rec/OpenKS-Expert-Rec/data/datav1/train_rel_is_principal_investigator_of.pkl
22.27108301357359
2223800
